## Run pipeline "pipeline_route" on Azure
### Note: need to obtain pipeline endpoint id from Azure and store in pipeline_id variable

### Imports

In [ ]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

from azureml.pipeline.core import PublishedPipeline
print("Pipeline SDK-specific imports completed")


# REST imports
from azureml.core.authentication import InteractiveLoginAuthentication
import requests
import json
print('REST specific imports completed')


### Parameters

In [ ]:
pipeline_id = 'a83e157d-3e84-42b3-aa8a-8a810da907c6' # '270023a9-93fd-481c-825b-944f0bb3f165' #'c25ab6d1-214c-401b-9871-f6dcc7afad11'

experiment_name = 'pipeline_route_REST'
param_input_data = './data'
param_ml_service = 'PAD'

### Load workspace, datastore

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config(path='azure_config_dev.json')
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')


## Get specific published pipeline

In [ ]:
published_pipeline = PublishedPipeline.get(ws, pipeline_id)
published_pipeline

## Run using REST

In [ ]:
auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

rest_endpoint = published_pipeline.endpoint

print("You can perform HTTP POST on URL {} to trigger this pipeline".format(rest_endpoint))

# specify the param when running the pipeline
response = requests.post(rest_endpoint, 
                         headers=aad_token, 
                         json={"ExperimentName": experiment_name,
                               "RunSource": "SDK",
                               "ParameterAssignments": {"input_data": param_input_data,
                                                        "ml_service": param_ml_service
                                                        #"workspace_name":"twv_test-ws"
                                                       }})


run_id = response.json()["Id"]

print(run_id)

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()
aad_token

In [ ]:
response.json()

In [ ]:
pipelines = PublishedPipeline.get_all(ws)

In [ ]:
pipelines